<a href="https://colab.research.google.com/github/Samarth1337/Vehicle_Classification_Image/blob/main/vgg16_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

## Transfer Learning using architecture of pre-trained model

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load the architecture of pretrained model
5. Train the model
6. Get predictions

## 1. Import neccessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Model
from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

## 2. Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# unzipping the Dataset.zip file
!unzip /content/drive/My\ Drive/Dataset.zip

In [ ]:
data = pd.read_csv('Dataset/emergency_classification.csv')

In [ ]:
X = []

for img_name in data.image_names:
    img = plt.imread('Dataset/images/' + img_name)
    X.append(img)
X = np.array(X)

In [ ]:
y = data.emergency_or_not.values

## 3. Pre-Process the data

Steps : 
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [ ]:
X = preprocess_input(X, mode='tf')

In [ ]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency
# with only one of them is true for every image
y = to_categorical(y)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load the architecture of pretrained model

In [ ]:
base_model = VGG16(weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
x = Dense(100, activation='relu', name='fc1')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='prediction')(x)
my_model = Model(input=base_model.input, output=y)
my_model.summary()

In [ ]:
my_model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

## 5. Train the model

In [ ]:
my_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

## 6. Get Predictions

In [ ]:
predictions = my_model.predict(X_valid)
predicted_class = np.argmax(predictions,axis=1)

In [ ]:
accuracy_score(y_valid[:,1], predicted_class)